In [133]:
# import modules
import ipywidgets as widgets
from IPython.display import display
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from ipyleaflet import (Map, Marker)
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pylab as pl
import operator

In [134]:
# load data base
poa_db = pd.read_csv('https://raw.githubusercontent.com/MarcoSCampos/testdata/master/poa_database.csv')
poa_nbh = pd.read_csv('https://raw.githubusercontent.com/MarcoSCampos/testdata/master/poa_nbh.csv')

In [135]:
#tiding the data
poa_db.rename(columns={'Bairro':'nbh','Area':'area', 'Valor':'value', 'Dormitorios':'rooms', 'Vagas':'parking'}, inplace=True)
poa_nbh.rename(columns={'bairro':'nbh'}, inplace=True)

#LabelEncoder
poa_nbh.insert(0, 'nbh_id', range(0, len(poa_nbh)))

# Merge with right join
poa_db2 = pd.merge(poa_nbh, poa_db, on='nbh', how='right')
poa_db2.head()
poa_db2['nbh2']=pd.Series(poa_db2.nbh, index=poa_db2.index).str.replace('_',' ')
poa_nbh['nbh2']=pd.Series(poa_nbh.nbh, index=poa_nbh.index).str.replace('_',' ')

# Generate a dict and sort
mydict=dict(zip(poa_nbh.nbh2,poa_nbh.nbh_id))
mydict=sorted(mydict.items(),key=operator.itemgetter(1))
#print (mydict)

In [136]:
w1 = widgets.IntSlider(description='rooms', min=1, max=4,step=1, value=2)
w2 = widgets.IntSlider(description='parking', min=0, max=2,step=1, value=1)
w3 = widgets.IntSlider(description='area',min=10, max=200,step=1, value=80)                       
w4=widgets.RadioButtons(description='Prediction:', options=['linear regression','random forest'], value='linear regression', disabled=False)
w5 = widgets.Dropdown(options=mydict,value=12,description='Neighborhood:',)
a=[-30.028591  ,-51.228060]
w6=Map(center=a, zoom=15)

In [ ]:
# modeling
a=False
if a:
 #Linear modeling
 #lm=smf.ols(formula='value ~ area + rooms + parking + nbh', data=poa_db).fit()
 lm=smf.ols(formula='value ~ area + rooms + parking + nbh', data=poa_db2).fit()
 new={'area':[50],'rooms':[2],'parking':[1], 'nbh':['Centro']}
 z=lm.predict(new)[0]
 print z
else:
 new1=pd.DataFrame({'area':[50],'rooms':[2],'parking':[1], 'nbh_id':[12]})
 cols = ['area', 'rooms', 'parking','nbh_id']
 rf = RandomForestRegressor(n_estimators=150)
 rf.fit(poa_db2[cols], poa_db2.value)
 z=rf.predict(new1[cols])[0]
 print z    

In [139]:
def place(c,d):
    e=d[['lat','lng']][c:c+1]
    e=list(e.values.flatten())
    return (e)

#a=place(w5.value,poa_nbh)

w6=Map(center=place(w5.value,poa_nbh), zoom=15)
mark=Marker(location=w6.center,title='Property value \n R$ 200000.00')
mark.visible
w6 += mark

#w6.interact(zoom=(5,15,15))


display (w1,w2,w3,w4,w5,w6)
